# Train mask over IOI edges and analyze mask vs known circuit

In [1]:
import os
os.chdir("/data/phillip_guo/circuit-breaking/ioi/")
from models import load_gpt2_weights, load_demo_gpt2, tokenizer
from data import retrieve_toxic_data, retrieve_owt_data, retrieve_toxic_data_low_loss, retrieve_toxic_filtered_data, FILTER_DEMO_LEN, CONTEXT_LENGTH
from inference import infer_batch_with_owt, infer_batch, prepare_fixed_demo, criterion
from torch.optim import AdamW
import torch
import pickle
import datasets
from tqdm import tqdm_notebook as tqdm
from itertools import cycle
# from eval import evaluate_model
from data import batch_text_to_tokens
import plotly.express as px

Using device: cuda:0


## Load mask into model

In [2]:
means_ioi = True

if means_ioi:
    with open("data/gpt2_ioi_abc_means.pkl", "rb") as f:
        means = pickle.load(f)[0]
else:
    with open("data/gpt2_means.pkl", "rb") as f:
        means = pickle.load(f)[0]

model = load_demo_gpt2(means=means)

In [3]:
means.shape

torch.Size([16, 157, 768])

In [4]:
template_type = "double"
alpha = 1

with open(f"models/alternative_necessary_masks_params_dict_lambda=1_{alpha=}_{means_ioi=}_{template_type=}.pkl", "rb") as f:
    necessary_masks_dict = pickle.load(f)
with open(f"models/alternative_sufficient_masks_params_dict_lambda=1_{alpha=}_{means_ioi=}_{template_type=}.pkl", "rb") as f:
    sufficient_masks_dict = pickle.load(f)
with open(f"models/params_dict_lambda=1_{means_ioi=}_{template_type=}.pkl", "rb") as f:
    mask_params_dict = pickle.load(f)
with open("models/acdcpp_mask_params.pkl", "rb") as f:
    acdcpp_mask_params = pickle.load(f)

final_masks = {"necessary": necessary_masks_dict[200], "sufficient": sufficient_masks_dict[200], "acdcpp": acdcpp_mask_params, "circuit_breaking": mask_params_dict[200]}

In [5]:
def load_mask_into_model(model, mask):
    # load in place
    mask_idx = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            param.data = mask[mask_idx].to(param.device)
            mask_idx += 1

def reset_mask(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            param.data = torch.ones_like(param.data).to(param.device)

## Test model before and after circuit breaking

In [6]:
import pickle
with open("data/ioi_prompts_single_template_test.pkl", "rb") as f:
    ioi_prompts_test = pickle.load(f)
    # ioi_sentences_test = [t[2] for t in ioi_sentences_test]

with open("data/eval_uniform.pkl", "rb") as f:
    uniform_samples = pickle.load(f)
    uniform_sentences = [t[2] for t in uniform_samples]

In [7]:
toxic_data_loader = retrieve_toxic_data(1, 16, tokenizer, tokenize=False, num_points=None, template_type=template_type)

In [8]:
for i, batch in enumerate(toxic_data_loader):
    print(batch)
    if i > 10:
        break

{'text': ['Then, Alicia and Kevin went to the station. Alicia gave a drink to Kevin']}
{'text': ['Then, Megan and Courtney went to the store. Courtney gave a kiss to Megan']}
{'text': ['Then, Mary and Danielle went to the restaurant. Danielle gave a snack to Mary']}
{'text': ['Then, Ryan and Kelly went to the hospital. Ryan gave a ring to Kelly']}
{'text': ['Then, Adam and Sara went to the house. Adam gave a kiss to Sara']}
{'text': ['Then, Scott and Travis went to the hospital. Travis gave a snack to Scott']}
{'text': ['Then, Kyle and Allison went to the restaurant. Kyle gave a ring to Allison']}
{'text': ['Then, Dustin and Timothy went to the station. Timothy gave a kiss to Dustin']}
{'text': ['Then, Jesse and Elizabeth went to the office. Elizabeth gave a necklace to Jesse']}
{'text': ['Then, Jason and Erin went to the garden. Erin gave a ring to Jason']}
{'text': ['Then, Sarah and Amy went to the house. Amy gave a computer to Sarah']}
{'text': ['Then, Daniel and Stephen went to the

In [9]:
ioi_prompts_test

[{'[PLACE]': 'house',
  '[OBJECT]': 'ring',
  'text': 'Then, Sarah and Amanda went to the house. Amanda gave a ring to Sarah',
  'IO': 'Sarah',
  'S': 'Amanda',
  'TEMPLATE_IDX': 0},
 {'text': 'Then, Amanda and Sarah went to the house. Sarah gave a ring to Amanda',
  'IO': 'Amanda',
  'S': 'Sarah',
  'TEMPLATE_IDX': 0},
 {'[PLACE]': 'garden',
  '[OBJECT]': 'bone',
  'text': 'Then, Christine and Kimberly went to the garden. Kimberly gave a bone to Christine',
  'IO': 'Christine',
  'S': 'Kimberly',
  'TEMPLATE_IDX': 0},
 {'text': 'Then, Kimberly and Christine went to the garden. Christine gave a bone to Kimberly',
  'IO': 'Kimberly',
  'S': 'Christine',
  'TEMPLATE_IDX': 0},
 {'[PLACE]': 'house',
  '[OBJECT]': 'bone',
  'text': 'Then, Nathan and Jason went to the house. Jason gave a bone to Nathan',
  'IO': 'Nathan',
  'S': 'Jason',
  'TEMPLATE_IDX': 0},
 {'text': 'Then, Jason and Nathan went to the house. Nathan gave a bone to Jason',
  'IO': 'Jason',
  'S': 'Nathan',
  'TEMPLATE_IDX':

In [10]:
# Run inference on an ioi_sentence
ioi_prompt = ioi_prompts_test[0]
print(ioi_prompt)

model.eval()
model.to('cuda')
def get_last_token(model, prompt, topk=5, sentence=False):
    # generate last token
    if not sentence:
        prompt_sentence = prompt['text']
    else:
        prompt_sentence = prompt

    tokens = tokenizer(prompt_sentence, return_tensors='pt').input_ids[:, :-1]

    # generate one token, decode original_model(ioi_tokens[:, :-1])
    model_outputs = model(tokens)[0]
    model_outputs = model_outputs.squeeze(0)[-1]
    probs = torch.nn.functional.softmax(model_outputs, dim=-1)

    topk_outputs = torch.topk(model_outputs, topk)
    topk_tokens = topk_outputs.indices
    topk_probs = probs[topk_outputs.indices]
    
    topk_tokens_decoded = tokenizer.batch_decode(topk_tokens)
    
    if not sentence:
        # Get logit diff by finding difference between logit of 
        io_token = tokenizer(" " + prompt['IO'], return_tensors='pt').input_ids[:, -1]
        s_token = tokenizer(" " + prompt['S'], return_tensors='pt').input_ids[:, -1]
        logit_diff = model_outputs[io_token][0] - model_outputs[s_token][0]
        return topk_tokens_decoded, topk_probs, logit_diff
    return topk_tokens_decoded, topk_probs

def get_ioi_score(model, num_samples):
    ave_logit_diffs = []
    for idx in range(num_samples):
        prompt = ioi_prompts_test[idx]
        ave_logit_diffs.append(get_last_token(model, prompt)[2])
    return sum(ave_logit_diffs) / len(ave_logit_diffs)

# get OWT loss
def get_owt_loss(model, num_samples):
    owt_losses = []
    for idx in range(num_samples):
        prompt = uniform_sentences[idx]
        owt_losses.append(infer_batch_with_owt(model, prompt['text'], prompt['IO'], prompt['S']))
    return sum(owt_losses) / len(owt_losses)

{'[PLACE]': 'house', '[OBJECT]': 'ring', 'text': 'Then, Sarah and Amanda went to the house. Amanda gave a ring to Sarah', 'IO': 'Sarah', 'S': 'Amanda', 'TEMPLATE_IDX': 0}


In [11]:
def invert_mask(mask, keep_output=True):
    new_mask = []
    for idx, m in enumerate(mask):
        if idx == 0 and keep_output:
            new_mask.append(torch.ones_like(m))
        else:
            new_mask.append(1 - m)
    return new_mask

In [12]:
reset_mask(model)
print("Average logit diff with no edges masked: ", get_ioi_score(model, 20))

load_mask_into_model(model, necessary_masks_dict[200])
print(f"Average logit diff with necessary edges masked: {get_ioi_score(model, 50)}")

load_mask_into_model(model, sufficient_masks_dict[200])
print(f"Average logit diff with sufficient edges masked: {get_ioi_score(model, 50)}")

load_mask_into_model(model, mask_params_dict[200])
print(f"Average logit diff post circuit-breaking: {get_ioi_score(model, 50)}")

load_mask_into_model(model, acdcpp_mask_params)
print(f"Average logit diff post masking ACDC++: {get_ioi_score(model, 50)}")

Average logit diff with no edges masked:  tensor(3.7131, device='cuda:0', grad_fn=<DivBackward0>)
Average logit diff with necessary edges masked: -0.3458968997001648
Average logit diff with sufficient edges masked: 18.609237670898438
Average logit diff post circuit-breaking: -0.4131331443786621
Average logit diff post masking ACDC++: -1.7468138933181763


In [13]:
acdcpp_mask_params

[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
         0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.], device='cuda:0'),
 tensor([[0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.]]),
 tensor([1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.]),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 

In [14]:
# load_mask_into_model(model, invert_mask(acdcpp_mask_params))
load_mask_into_model(model, final_masks['sufficient'])
with torch.no_grad():
    test_ioi_sentences = ["While Alicia and Joshua were commuting to the restaurant, Joshua gave a snack to", "While Joshua and Alicia were commuting to the restaurant, Joshua gave a snack to"]
    for test_ioi_sentence in test_ioi_sentences:
        correct_token_id = tokenizer.encode(" Alicia", return_tensors="pt").squeeze().item()
        other_token_id = tokenizer.encode(" Joshua", return_tensors="pt").squeeze().item()
        test_ioi_tokens = tokenizer.encode(test_ioi_sentence, return_tensors="pt").to('cuda')
        generation = model(test_ioi_tokens)[0][:, -1]
        probs = torch.softmax(generation, dim=-1)
        print(f"Best Token: {tokenizer.batch_decode(torch.argmax(generation, dim=-1))}, P(Alicia) = {probs[:,correct_token_id].item()}, logit diff = {generation[:,correct_token_id].item() - generation[:,other_token_id].item()}")

Best Token: [' Alicia'], P(Alicia) = 0.8985728621482849, logit diff = 2.338129997253418
Best Token: [' Joshua'], P(Alicia) = 2.0893772045837977e-08, logit diff = -17.600988388061523


In [10]:
print("Inverted Masks")
reset_mask(model)
print("Average logit diff with no edges masked: ", get_ioi_score(model, 20))

load_mask_into_model(model, invert_mask(necessary_masks_dict[200]))
print(f"Average logit diff with necessary edges masked: {get_ioi_score(model, 100)}")

load_mask_into_model(model, invert_mask(sufficient_masks_dict[200]))
print(f"Average logit diff with sufficient edges masked: {get_ioi_score(model, 100)}")

load_mask_into_model(model, invert_mask(mask_params_dict[200]))
print(f"Average logit diff post circuit-breaking: {get_ioi_score(model, 100)}")

load_mask_into_model(model, invert_mask(acdcpp_mask_params))
print(f"Average logit diff post masking ACDC++: {get_ioi_score(model, 100)}")

Inverted Masks
Average logit diff with no edges masked:  tensor(2.9523, device='cuda:0', grad_fn=<DivBackward0>)
Average logit diff with necessary edges masked: -0.0035178374964743853
Average logit diff with sufficient edges masked: -1.110324740409851
Average logit diff post circuit-breaking: 0.003694457933306694
Average logit diff post masking ACDC++: 0.06923042237758636


In [13]:
import nvsmi

nvsmi.get_gpus()
nvsmi.get_available_gpus()
nvsmi.get_gpu_processes()

[pid: 187963 | gpu_id: 0 | gpu_uuid: GPU-23af42b5-2478-0724-7712-899809c890ac | gpu_name: NVIDIA A100-SXM4-80GB | used_memory:  9212.0MB,
 pid: 188436 | gpu_id: 0 | gpu_uuid: GPU-23af42b5-2478-0724-7712-899809c890ac | gpu_name: NVIDIA A100-SXM4-80GB | used_memory:  5264.0MB,
 pid: 230888 | gpu_id: 0 | gpu_uuid: GPU-23af42b5-2478-0724-7712-899809c890ac | gpu_name: NVIDIA A100-SXM4-80GB | used_memory: 27400.0MB]

## Manually Check Edges

In [42]:
name_mover_heads = ['a10.0', 'a9.9', 'a9.6']
negative_heads = ['a10.7', 'a11.10']
s2_inhibition_heads = ['a8.10', 'a7.9', 'a8.6', 'a7.3']
induction_heads = ['a5.5', 'a6.9', 'a5.9', 'a5.8']
duplicate_token_heads = ['a0.1', 'a0.10', 'a3.0']
previous_token_heads = ['a4.11', 'a2.2', 'a2.9']
backup_name_mover_heads = ['a11.2', 'a10.2', 'a10.6', 'a10.1', 'a10.10', 'a9.7', 'a11.9', 'a11.3']

circuit_dict = {}

for head in name_mover_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 'name_mover'

for head in negative_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 'negative'

for head in s2_inhibition_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 's2_inhibition'

for head in induction_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 'induction'

for head in duplicate_token_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 'duplicate_token'

for head in previous_token_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 'previous_token'

for head in backup_name_mover_heads:
    circuit_dict[(int(head.split('.')[0][1:]), head)] = 'backup_name_mover'

In [43]:
def check_relevant_edges(mask_edges, circuit_nodes=circuit_dict):
    # edges where one of the nodes is in the circuit
    relevant_edges = []

    # edges where both nodes are in the circuit
    circuit_edges = []
    
    for edge in mask_edges:
        if edge[0] in circuit_nodes or edge[1] in circuit_nodes:
            relevant_edges.append(edge)
        if edge[0] in circuit_nodes and edge[1] in circuit_nodes:
            circuit_edges.append(edge)
    return relevant_edges, circuit_edges

In [44]:
from mask_utils import get_nodes_and_edges

for name, mask in final_masks.items():
    print(name)
    _, _, edges, _ = get_nodes_and_edges(mask, edge_0=(False if name == "sufficient" else True))
    relevant_edges, circuit_edges = check_relevant_edges(edges)
    print(f"For {name}, number of relevant edges: {len(relevant_edges)}, number of circuit edges: {len(circuit_edges)}, total number of edges: {len(edges)}")
# _, _, sufficient_edges, _ = get_nodes_and_edges(necessary_masks_dict[200])
# relevant_edges, circuit_edges = check_relevant_edges(sufficient_edges)
# print(f"number of relevant edges: {len(relevant_edges)}, number of circuit edges: {len(circuit_edges)}")

# _, _, acdcpp_edges, _ = get_nodes_and_edges(acdcpp_mask_params)
# relevant_edges, circuit_edges = check_relevant_edges(acdcpp_edges)
# print(f"number of relevant edges: {len(relevant_edges)}, number of circuit edges: {len(circuit_edges)}")

necessary
For necessary, number of relevant edges: 116, number of circuit edges: 6, total number of edges: 491
sufficient
For sufficient, number of relevant edges: 32, number of circuit edges: 4, total number of edges: 55
acdcpp
For acdcpp, number of relevant edges: 3449, number of circuit edges: 258, total number of edges: 11379
circuit_breaking
For circuit_breaking, number of relevant edges: 22, number of circuit edges: 0, total number of edges: 176


In [45]:
for epochs_trained in sufficient_masks_dict:
    _, _, sufficient_edges, _ = get_nodes_and_edges(sufficient_masks_dict[epochs_trained], edge_0=False)
    relevant_edges, circuit_edges = check_relevant_edges(sufficient_edges)
    if len(circuit_edges) > 0:
        print(f"{epochs_trained} epochs trained, number of relevant edges: {len(relevant_edges)}, number of circuit edges: {len(circuit_edges)}")

10 epochs trained, number of relevant edges: 182, number of circuit edges: 17
20 epochs trained, number of relevant edges: 151, number of circuit edges: 16
30 epochs trained, number of relevant edges: 23, number of circuit edges: 5
40 epochs trained, number of relevant edges: 30, number of circuit edges: 6
50 epochs trained, number of relevant edges: 100, number of circuit edges: 18
60 epochs trained, number of relevant edges: 19, number of circuit edges: 4
70 epochs trained, number of relevant edges: 3, number of circuit edges: 1
80 epochs trained, number of relevant edges: 15, number of circuit edges: 2
90 epochs trained, number of relevant edges: 14, number of circuit edges: 5
100 epochs trained, number of relevant edges: 39, number of circuit edges: 5
110 epochs trained, number of relevant edges: 9, number of circuit edges: 3
130 epochs trained, number of relevant edges: 7, number of circuit edges: 1
140 epochs trained, number of relevant edges: 8, number of circuit edges: 1
150 ep

In [47]:
# Get overlaps between each edge set
edges_dict = {"necessary": get_nodes_and_edges(necessary_masks_dict[200], edge_0=True)[2], "sufficient": get_nodes_and_edges(sufficient_masks_dict[200], edge_0=False)[2], "acdcpp": get_nodes_and_edges(acdcpp_mask_params, edge_0=False)[2], "circuit_breaking": get_nodes_and_edges(mask_params_dict[200], edge_0=True)[2]}

# make a 4x4 matrix of intersections between the sets of edges



In [51]:
import pandas as pd

# Assuming you already have the sets from your dictionary
# For demonstration, let's consider these are your sets:
necessary = edges_dict["necessary"]
sufficient = edges_dict["sufficient"]
acdcpp = edges_dict["acdcpp"]
circuit_breaking = edges_dict["circuit_breaking"]

# List of all sets
sets = [necessary, sufficient, acdcpp, circuit_breaking]
set_names = ["necessary", "sufficient", "acdcpp", "circuit_breaking"]

# Initialize an empty matrix
matrix = []

# Calculate the intersections
for set1 in sets:
    row = []
    for set2 in sets:
        # Append the length of intersection to the row
        row.append(len(set1.intersection(set2)))
    # Append the row to the matrix
    matrix.append(row)

# Create a DataFrame from the matrix
df = pd.DataFrame(matrix, index=set_names, columns=set_names)

# Display the DataFrame
df

,necessary,sufficient,acdcpp,circuit_breaking
necessary,491,15,37,46
sufficient,15,55,26,3
acdcpp,37,26,232,13
circuit_breaking,46,3,13,176


In [53]:
# Initialize an empty matrix
matrix = []

# Calculate the intersections
for set1 in sets:
    row = []
    for set2 in sets:
        # Append the length of intersection to the row
        row.append(len(set1.intersection(set2)) / len(set1) * 100)
    # Append the row to the matrix
    matrix.append(row)

# Create a DataFrame from the matrix
proportional_df = pd.DataFrame(matrix, index=set_names, columns=set_names)

# Display the DataFrame
proportional_df

,necessary,sufficient,acdcpp,circuit_breaking
necessary,100.000000,3.054990,7.535642,9.368635
sufficient,27.272727,100.000000,47.272727,5.454545
acdcpp,15.948276,11.206897,100.000000,5.603448
circuit_breaking,26.136364,1.704545,7.386364,100.000000


## Visualize mask
Create the computational graphs in edge attribution patching paper

### Load mask and calculate what edges are present

In [17]:
# get param names
param_names = []
for name, param in model.named_parameters():
    if param.requires_grad:
        param_names.append(name)

def get_nodes_and_edges(mask_params, edge_0=True):
    """
    If edge_0 is True, then edges are between nodes with mask value 0. Else, edges are between nodes with mask value 1.
    """
    # calculate which nodes will be in the graph
    connected_nodes = set()
    # add embed node at position
    # connected_nodes.add((-1, "embed"))
    n_heads = 12
    n_layers = 12

    # associate each node with a position
    all_possible_nodes = [(-1, "embed")]
    mask_dict = {}
    # empty tensor
    mask_dict["embed"] = torch.zeros(size=(0,))
    for idx in range(len(mask_params)):
        if "attention" in param_names[idx]:
            layer = int(param_names[idx].split(".")[1])
            for i in range(n_heads):
                all_possible_nodes.append((layer, f"a{layer}.{i}"))
                mask_dict[f"a{layer}.{i}"] = mask_params[idx][:,i].detach().cpu()
        elif "mlp" in param_names[idx]:
            layer = int(param_names[idx].split(".")[1])
            all_possible_nodes.append((layer, f"m{layer}"))
            mask_dict[f"m{layer}"] = mask_params[idx].detach().cpu()
    all_possible_nodes.append((n_heads, "output"))
    mask_dict["output"] = mask_params[-1]

    # Calculate where edges are based on the mask
    # Edge between node i and node j if mask_dict[i][all_possible_nodes.index(j)] == 0
    edges = set()
    for i in range(len(all_possible_nodes)):
        for j in range(len(all_possible_nodes)):
            j_index = all_possible_nodes.index(all_possible_nodes[j])
            if j_index < len(mask_dict[all_possible_nodes[i][1]]) and mask_dict[all_possible_nodes[i][1]][all_possible_nodes.index(all_possible_nodes[j])] == (0 if edge_0 else 1):
                edges.add((all_possible_nodes[i], all_possible_nodes[j]))
    
    nodes_with_edges = set([node for edge in edges for node in edge])

    return all_possible_nodes, nodes_with_edges, edges, mask_dict

In [18]:
all_possible_nodes, nodes_with_edges, edges, mask_dict = get_nodes_and_edges(final_masks["necessary"], edge_0=False)

### Analyze ACDC and Compare
I separately used ACDC++ (EAP from "Attribution Patching Outperforms Automated Circuit Discovery" paper) to get the known circuit edges. I want to compare my various learned masks (from different losses) to the known circuit edges.

In [20]:
with open("models/acdcpp_edges_original.pkl", "rb") as f:
    acdcpp_edges_long = pickle.load(f)
acdcpp_edges_long

({0.08: {'blocks.0.attn.hook_k[:, :, 10]blocks.0.hook_k_input[:, :, 10]',
   'blocks.0.attn.hook_q[:, :, 10]blocks.0.hook_q_input[:, :, 10]',
   'blocks.0.attn.hook_q[:, :, 2]blocks.0.hook_q_input[:, :, 2]',
   'blocks.0.attn.hook_result[:, :, 10]blocks.0.attn.hook_k[:, :, 10]',
   'blocks.0.attn.hook_result[:, :, 10]blocks.0.attn.hook_q[:, :, 10]',
   'blocks.0.attn.hook_result[:, :, 10]blocks.0.attn.hook_v[:, :, 10]',
   'blocks.0.attn.hook_result[:, :, 1]blocks.0.attn.hook_k[:, :, 1]',
   'blocks.0.attn.hook_result[:, :, 1]blocks.0.attn.hook_q[:, :, 1]',
   'blocks.0.attn.hook_result[:, :, 1]blocks.0.attn.hook_v[:, :, 1]',
   'blocks.0.attn.hook_result[:, :, 3]blocks.0.attn.hook_k[:, :, 3]',
   'blocks.0.attn.hook_result[:, :, 3]blocks.0.attn.hook_q[:, :, 3]',
   'blocks.0.attn.hook_result[:, :, 3]blocks.0.attn.hook_v[:, :, 3]',
   'blocks.0.attn.hook_result[:, :, 6]blocks.0.attn.hook_k[:, :, 6]',
   'blocks.0.attn.hook_result[:, :, 6]blocks.0.attn.hook_q[:, :, 6]',
   'blocks.0.att

In [28]:
# acdcpp edges are in format 'blocks.1.attn.hook_result[:, :, 10]blocks.0.hook_mlp_in[:]', convert to format of ((1, 'a1.10'), (0, 'm0'))

def get_node_name(node_name, show_full_index=False):
    """Node name for use in pretty graphs"""

    def get_index(node_name_long):
        # Get the index by looking for number in brackets
        # e.g. blocks.1.attn.hook_result[:, :, 10] -> 10
        index = node_name_long.split("[")[-1].split("]")[0]
        index = index.split(", ")[-1]
        return int(index)

    if not show_full_index:
        name = ""
        qkv_substrings = [f"hook_{letter}" for letter in ["q", "k", "v"]]
        qkv_input_substrings = [f"hook_{letter}_input" for letter in ["q", "k", "v"]]

        # Handle embedz
        if "resid_pre" in node_name:
            assert "0" in node_name and not any([str(i) in node_name for i in range(1, 10)])
            name += "embed"
            layer = -1
            # if len(node.index.hashable_tuple) > 2:
            #     name += f"_[{node.index.hashable_tuple[2]}]"
            # return name

        elif "embed" in node_name:
            # name = "pos_embeds" if "pos" in node_name else "token_embeds"
            name = "embed"
            layer = -1

        # Handle q_input and hook_q etc
        elif any([node_name.endswith(qkv_input_substring) for qkv_input_substring in qkv_input_substrings]):
            relevant_letter = None
            for letter, qkv_substring in zip(["q", "k", "v"], qkv_substrings):
                if qkv_substring in node_name:
                    assert relevant_letter is None
                    relevant_letter = letter
            name += "a" + node_name.split(".")[1] + "." + str(get_index(node_name)) + "_" + relevant_letter
            layer = int(node_name.split(".")[1])

        # Handle attention hook_result
        elif "hook_result" in node_name or any([qkv_substring in node_name for qkv_substring in qkv_substrings]):
            name = "a" + node_name.split(".")[1] + "." + str(get_index(node_name))
            layer = int(node_name.split(".")[1])

        # Handle MLPs
        elif node_name.endswith("resid_mid"):
            raise ValueError("We removed resid_mid annotations. Call these mlp_in now.")
        elif "mlp" in node_name:
            name = "m" + node_name.split(".")[1]
            layer = int(node_name.split(".")[1])

        # Handle resid_post
        elif "resid_post" in node_name:
            name += "output"
            layer = 12

        # elif "mlp" in node_name:
        #     name += "m" + node_name.split(".")[1]
        else:
            raise ValueError(f"Unrecognized node name {node_name}")

    else:
        name = node_name
        # name = node_name + str(node.index.graphviz_index(use_actual_colon=True))

    # get layer by looking for number before first dot
    

    return layer, name

acdcpp_edges = set()
for edge in acdcpp_edges_long[0][0.08]:
    # split the edge into two nodes, e.g. blocks.1.attn.hook_result[:, :, 10]blocks.0.hook_mlp_in[:] into blocks.1.attn.hook_result[:, :, 10] and blocks.0.hook_mlp_in[:]
    node_1 = get_node_name(edge.split("]")[0]+"]", show_full_index=False)
    node_2 = get_node_name(edge.split("]")[1]+"]", show_full_index=False)
    if node_1 != node_2:
        acdcpp_edges.add((node_1, node_2))


In [31]:
acdcpp_edges

{((0, 'a0.1'), (-1, 'embed')),
 ((0, 'a0.10'), (-1, 'embed')),
 ((0, 'a0.3'), (-1, 'embed')),
 ((0, 'a0.5'), (-1, 'embed')),
 ((0, 'm0'), (-1, 'embed')),
 ((0, 'm0'), (0, 'a0.1')),
 ((0, 'm0'), (0, 'a0.10')),
 ((0, 'm0'), (0, 'a0.4')),
 ((1, 'm1'), (0, 'm0')),
 ((2, 'a2.11'), (0, 'm0')),
 ((2, 'm2'), (-1, 'embed')),
 ((2, 'm2'), (0, 'a0.10')),
 ((2, 'm2'), (0, 'm0')),
 ((2, 'm2'), (1, 'a1.11')),
 ((3, 'a3.0'), (0, 'm0')),
 ((3, 'a3.0'), (1, 'm1')),
 ((3, 'a3.0'), (2, 'm2')),
 ((3, 'm3'), (0, 'm0')),
 ((3, 'm3'), (2, 'm2')),
 ((3, 'm3'), (3, 'a3.0')),
 ((4, 'm4'), (0, 'm0')),
 ((4, 'm4'), (1, 'a1.11')),
 ((4, 'm4'), (1, 'm1')),
 ((4, 'm4'), (2, 'a2.2')),
 ((4, 'm4'), (2, 'a2.9')),
 ((4, 'm4'), (2, 'm2')),
 ((4, 'm4'), (3, 'a3.0')),
 ((4, 'm4'), (3, 'a3.10')),
 ((4, 'm4'), (3, 'a3.3')),
 ((4, 'm4'), (3, 'm3')),
 ((4, 'm4'), (4, 'a4.11')),
 ((4, 'm4'), (4, 'a4.3')),
 ((4, 'm4'), (4, 'a4.4')),
 ((4, 'm4'), (4, 'a4.7')),
 ((5, 'a5.5'), (0, 'm0')),
 ((5, 'a5.5'), (2, 'a2.2')),
 ((5, 'a5.5'),

### Analyze overlaps between different edges

In [ ]:
with open("models/alternative_necessary_masks_params_dict_lambda=1.pkl", "rb") as f:
    alternative_necessary_mask_params = pickle.load(f)
    alternative_necessary_mask_params = alternative_necessary_mask_params[200]
with open("models/alternative_sufficient_masks_params_dict_lambda=1.pkl", "rb") as f:
    alternative_sufficient_mask_params = pickle.load(f)
    alternative_sufficient_mask_params = alternative_sufficient_mask_params[200]
_, _, alternative_necessary_edges, _ = get_nodes_and_edges(alternative_necessary_mask_params)
_, _, alternative_sufficient_edges, _ = get_nodes_and_edges(alternative_sufficient_mask_params, edge_0=False)

In [ ]:
get_nodes_and_edges(alternative_sufficient_mask_params, edge_0=False)

In [ ]:
print(f"{len(edges)=}, {len(acdcpp_edges)=}, {len(edges.intersection(acdcpp_edges))=}")
print(edges.intersection(acdcpp_edges))

In [ ]:
# Get overlaps between all edges (regular edges, necessary, sufficient, acdcpp) (make a table with tabulate)
edges_dict = {"circuit_breaking":edges, "ioi_necessary":alternative_necessary_edges, "ioi_sufficient":alternative_sufficient_edges, "acdcpp":acdcpp_edges}
for edge_type in edges_dict:
    for second_edge_type in edges_dict:
        print(f"{edge_type} and {second_edge_type}: {len(edges_dict[edge_type].intersection(edges_dict[second_edge_type]))} edges in common")#, {edges_dict[edge_type].intersection(edges_dict[second_edge_type])}")

### Visualizations

In [ ]:
def create_aligned_graph(all_possible_nodes, edges):
    G = pgv.AGraph(strict=False, directed=True)

    # Find the maximum layer number for adjusting the graph
    max_layer = max(layer for layer, _ in all_possible_nodes if isinstance(layer, int))
    nodes_with_edges = set([node for edge in edges for node in edge])

    # Add nodes and edges to the graph
    for node in all_possible_nodes:
        if node in [edge[0] for edge in edges] or node in [edge[1] for edge in edges]:
            G.add_node(node[1], layer=str(max_layer - node[0]))

    for edge in edges:
        G.add_edge(edge[1][1], edge[0][1])

    # Create subgraphs to ensure nodes of the same layer have the same rank
    for layer in range(max_layer, -2, -1):
        with G.subgraph(name=f'cluster_{layer}') as s:
            s.graph_attr['rank'] = 'same'
            for node in nodes_with_edges:
                if node[0] == layer:
                    s.add_node(node[1])

    # Apply layout and render the graph
    G.layout(prog='dot')
    G.draw('aligned_graph.png')
    return Image('aligned_graph.png')

# Call the function with your nodes and edges
flipped_graph_image = create_aligned_graph(all_possible_nodes, edges)

# To display the graph in Jupyter Notebook
flipped_graph_image


In [ ]:
# intersecting edges graph
for edge_type in edges_dict:
    for second_edge_type in edges_dict:
        if edge_type == second_edge_type:
            continue
        # make a graph with just the intersecting edges, title it with the two edge types
        print(f"Intersection between {edge_type} and {second_edge_type}: {len(edges_dict[edge_type].intersection(edges_dict[second_edge_type]))} edges in common, {edges_dict[edge_type].intersection(edges_dict[second_edge_type])}")
        
        intersecting_edges_graph = create_aligned_graph(all_possible_nodes, edges_dict[edge_type].intersection(edges_dict[second_edge_type]))
        display(intersecting_edges_graph)
        # intersecting_edges_graph.render(f"intersecting_edges_graph_{edge_type}_{second_edge_type}", format="png", cleanup=True)
# intersecting_edges_graph = create_aligned_graph(all_possible_nodes, edges.intersection(acdcpp_edges))
# intersecting_edges_graph

In [ ]:
import pygraphviz as pgv
from pathlib import Path
from IPython.display import Image

def show(nodes, edges, fname=None):
    g = pgv.AGraph(strict=True, directed=True)
    g.graph_attr.update(ranksep='0.1', nodesep='0.1', compound=True)
    g.node_attr.update(fixedsize='true', width='1.5', height='.5')
    
    layer_to_subgraph = {}

    # Create a subgraph for each layer
    for node in nodes:
        layer = node[0]
        if layer not in layer_to_subgraph:
            # Each layer has its own subgraph with 'rank=same' to ensure they are on the same level
            layer_to_subgraph[layer] = g.add_subgraph(name=f'cluster_{layer}', rank='same')
            
        # Here you add the node to the appropriate subgraph
        layer_to_subgraph[layer].add_node(node, label=str(node[1]))

    # Now, add the edges to the graph
    for edge in edges:
        g.add_edge(edge[0], edge[1])
    
    # If a filename is provided, write the file and optionally render to an image
    if fname:
        fpath = Path(fname)
        base_fname = fpath.stem
        base_path = fpath.parent
        base_path.mkdir(exist_ok=True, parents=True)
        
        # Write the dot file
        g.write(path=base_path / f"{base_fname}.gv")
        
        # Render to an image
        g.layout(prog='dot')
        g.draw(path=base_path / f"{base_fname}.png")
        
    return g


g = show(nodes_with_edges, edges, fname="graph.gv")
Image(g.draw(format='png', prog='dot'))